<a href="https://colab.research.google.com/github/jlacy1/Jaylen_DTSC3020_Fall2025/blob/main/jwl0158_Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [6]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [7]:
# Q1 — Write your answer here

# First step is to Read and extract the table
def q1_read_table(html: str) -> pd.DataFrame:
    tables = pd.read_html(html)
    for table in tables:
        if table.shape[1] > 3:
            df = flatten_headers(table)
            return df
    raise ValueError("No suitable table found with > 3 columns.")

# Next we clean the data
def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    if 'Alpha-2 code' in df.columns:
        df['Alpha-2 code'] = df['Alpha-2 code'].str.upper()
    if 'Alpha-3 code' in df.columns:
        df['Alpha-3 code'] = df['Alpha-3 code'].str.upper()

    if 'Numeric' in df.columns:
        df['Numeric'] = pd.to_numeric(df['Numeric'], errors='coerce').astype('Int64')
        df = df.dropna(subset=['Numeric'])

    return df

# We then Sort and return Top-N
def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return df.sort_values(by='Numeric', ascending=False).head(top)


html = fetch_html("https://www.iban.com/country-codes")
df_raw = q1_read_table(html)
df_clean = q1_clean(df_raw)
df_top15 = q1_sort_top(df_clean)

# Save and preview
df_clean.to_csv("data_q1.csv", index=False)
print("Top 15 countries by Numeric Code:")
print(df_top15)

Top 15 countries by Numeric Code:
                                               Country Alpha-2 code  \
247                                             Zambia           ZM   
246                                              Yemen           YE   
192                                              Samoa           WS   
244                                  Wallis and Futuna           WF   
240                 Venezuela (Bolivarian Republic of)           VE   
238                                         Uzbekistan           UZ   
237                                            Uruguay           UY   
35                                        Burkina Faso           BF   
243                              Virgin Islands (U.S.)           VI   
236                     United States of America (the)           US   
219                       Tanzania, United Republic of           TZ   
108                                        Isle of Man           IM   
113                                        

/tmp/ipython-input-1311215530.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)
/tmp/ipython-input-1311215530.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [8]:
# Q2 — Write your answer here

def q2_parse_items(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "lxml")
    items = soup.select("tr.athing")
    records = []

    for item in items:
        rank_tag = item.select_one("span.rank")
        title_tag = item.select_one("a.storylink")
        subtext = item.find_next_sibling("tr").select_one("td.subtext")

        rank = rank_tag.text.strip().replace(".", "") if rank_tag else ""
        title = title_tag.text.strip() if title_tag else ""
        link = title_tag["href"] if title_tag and title_tag.has_attr("href") else ""

        points_tag = subtext.select_one("span.score")
        points = points_tag.text.strip().replace(" points", "") if points_tag else ""

        user_tag = subtext.select_one("a.hnuser")
        user = user_tag.text.strip() if user_tag else ""

        comments_tag = subtext.find_all("a")[-1]
        comments_text = comments_tag.text.strip() if comments_tag else ""
        comments = comments_text.replace("\xa0comments", "").replace("comment", "").strip() if "comment" in comments_text else ""

        records.append({
            "rank": rank,
            "title": title,
            "link": link,
            "points": points,
            "comments": comments,
            "user": user
        })

    return pd.DataFrame(records)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    for col in ["rank", "points", "comments"]:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

    for col in ["title", "link", "user"]:
        df[col] = df[col].fillna("").astype(str).str.strip()

    return df

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return df.sort_values(by="points", ascending=False).head(top)

html = fetch_html("https://news.ycombinator.com/")
df_raw = q2_parse_items(html)
df_clean = q2_clean(df_raw)
df_top15 = q2_sort_top(df_clean)

# Save and preview
df_clean.to_csv("data_q2.csv", index=False)
print("Top 15 Hacker News stories by points:")
print(df_top15)



Top 15 Hacker News stories by points:
    rank title link  points  comments         user
28    29                935       372  tabletcorry
4      5                606       142     saikatsg
26    27                531       216       costco
7      8                506       174       erhuve
17    18                485       382      donohoe
27    28                456       138        vrnvu
14    15                195        70     agluszak
2      3                194       109        art-w
5      6                165        93         c420
0      1                161        54     PhilKunz
23    24                133        38       ibobev
20    21                133       113      pretext
16    17                116        91    lawnchair
11    12                103        30       jarbus
1      2                 86        32       speckx
